<a href="https://colab.research.google.com/github/LI3ARA/FYIRP/blob/main/Notebooks/25/VQA_RAG/Googl_colab/host_llm_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working code

Colab:

- Run llava-cli in a Flask or FastAPI app

- Expose it using ngrok so VS Code can reach it

Steps: 
- Load the LLaVA model using llava.cpp

- Start a Flask server in Colab

- Expose the server via ngrok

- Accept an image + prompt, run llava-cli, and return the output

In [1]:
# Install dependencies
!pip install flask llama-cpp-python --quiet
!wget -q -O ngrok.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok.tgz
!chmod +x ngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 32.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
# Add your ngrok auth token (looks like 2N...)
!./ngrok config add-authtoken 2vLvftpzesWBGHU32SAnWOWmryA_CvhQGZ7cyfpk6TEkPG1b

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
import subprocess, time, requests
from llama_cpp import Llama
from flask import Flask, request, jsonify

In [6]:
# Start ngrok on port 5000 and fetch public URL

ngrok_proc = subprocess.Popen(['./ngrok', 'http', '5000'])
time.sleep(5)
try:
    res = requests.get('http://localhost:4040/api/tunnels')
    url = res.json()['tunnels'][0]['public_url']
    print('✅ Public URL:', url)
except Exception as e:
    print('❌ Could not get ngrok URL:', e)

✅ Public URL: https://0232-34-16-171-130.ngrok-free.app


In [7]:
# Load model
llm = Llama.from_pretrained(
    repo_id='TheBloke/Llama-2-13B-chat-GGUF',
    filename='llama-2-13b-chat.Q3_K_S.gguf',
    n_gpu_layers=40,
    n_ctx=2048
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q3_K_S.gguf:   0%|          | 0.00/5.66G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/./llama-2-13b-chat.Q3_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimensio

In [ ]:
# Start Flask app with LLaMA endpoint

app = Flask(__name__)

@app.route('/')
def home():
    return '✅ LLaMA Flask API is running! Use /generate'

@app.route('/generate', methods=['POST'])
def generate():
    prompt = request.json.get('prompt', '')
    if not prompt:
        return jsonify({'error': 'Prompt is required'}), 400
    output = llm(prompt, max_tokens=200)
    return jsonify({'response': output['choices'][0]['text'].strip()})

app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
llama_perf_context_print:        load time =     602.34 ms
llama_perf_context_print: prompt eval time =     602.16 ms /     4 tokens (  150.54 ms per token,     6.64 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     603.06 ms /     5 tokens
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 13:11:07] "POST /generate HTTP/1.1" 200 -
Llama.generate: 3 prefix-match hit, remaining 1 prompt tokens to eval
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 13:12:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 13:12:13] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 13:12:19] "GET /generate HTTP/1.1" 405 -
llama_perf_contex



---



# Step by step working- test code

Without llm , just using a test text to pass

## In colab

In [4]:
!pip install ngrok


In [5]:
!wget -q -O ngrok.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok.tgz
!chmod +x ngrok

# 🧠 Your ngrok auth token goes here (not your API key)
!./ngrok config add-authtoken <here>


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [21]:
! pip install llama_cpp

ERROR: Could not find a version that satisfies the requirement llama_cpp (from versions: none)
ERROR: No matching distribution found for llama_cpp


---

In [6]:
import subprocess, time, requests
from flask import Flask

# Start ngrok on port 5000
ngrok_proc = subprocess.Popen(["./ngrok", "http", "5000"])
time.sleep(5)

# Try to get the public tunnel URL
try:
    tunnels = requests.get("http://localhost:4040/api/tunnels").json()
    public_url = tunnels["tunnels"][0]["public_url"]
    print("✅ ngrok public URL:", public_url)
except Exception as e:
    print("❌ ngrok failed:", e)


✅ ngrok public URL: https://d67b-104-199-189-124.ngrok-free.app


In [7]:
app = Flask(__name__)

@app.route("/")
def index():
    return "✅ LLaMA Flask API is running!"

app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [18]:
# Check if ngrok is alive
import subprocess, time, requests

# Start ngrok (it runs in the background)
ngrok_proc = subprocess.Popen(["./ngrok", "http", "5000"])
time.sleep(4)

# Try to read the tunnel info
try:
    res = requests.get("http://localhost:4040/api/tunnels")
    print("🔍 Raw response:", res.text)
    data = res.json()
    print("✅ Public URL:", data["tunnels"][0]["public_url"])
except Exception as e:
    print("❌ Could not fetch ngrok tunnel:", e)


🔍 Raw response: {"tunnels":[{"name":"command_line","ID":"e2a1c3a7137a897877e63bb2f6a64125","uri":"/api/tunnels/command_line","public_url":"https://f4cf-104-199-189-124.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}

✅ Public URL: https://f4cf-104-199-189-124.ngrok-free.app


In [19]:
requests.get("http://localhost:4040/api/tunnels").json()

{'tunnels': [{'name': 'command_line',
   'ID': 'e2a1c3a7137a897877e63bb2f6a64125',
   'uri': '/api/tunnels/command_line',
   'public_url': 'https://f4cf-104-199-189-124.ngrok-free.app',
   'proto': 'https',
   'config': {'addr': 'http://localhost:5000', 'inspect': True},
   'metrics': {'conns': {'count': 0,
     'gauge': 0,
     'rate1': 0,
     'rate5': 0,
     'rate15': 0,
     'p50': 0,
     'p90': 0,
     'p95': 0,
     'p99': 0},
    'http': {'count': 0,
     'rate1': 0,
     'rate5': 0,
     'rate15': 0,
     'p50': 0,
     'p90': 0,
     'p95': 0,
     'p99': 0}}}],
 'uri': '/api/tunnels'}

In [20]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/")
def home():
    return "✅ Hello from Flask with ngrok!"

@app.route("/generate", methods=["POST"])
def generate():
    data = request.get_json()
    prompt = data.get("prompt", "No prompt provided.")
    return jsonify({
        "response": f"🧪 Test response to: '{prompt}'"
    })

app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 12:23:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 12:23:44] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 12:24:24] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2025 12:29:44] "GET / HTTP/1.1" 200 -


## In VS code

In [1]:
# In vs code
import requests

url = "https://f4cf-104-199-189-124.ngrok-free.app/generate"  # Replace with your real one
response = requests.post(url, json={"prompt": "Hello world!"})
print("🧠 Response:", response.json()["response"])



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

---

In [3]:
! pip install pyngrok

In [4]:
# Use ngrok to expose your Flask server
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("Public URL:", public_url)


t=2025-04-16T10:17:29+0530 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2025-04-16T10:17:29+0530 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2025-04-16T10:17:29+0530 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.